# Scheme III

## Warm-Up
- Write a function that takes in two lists, and returns the one that is longer


## Recursion Refresher
- Borrowed from CMSC 201 Slides:
    - Break problem into smaller pieces
    - Have a base case that permits us to stop
    - All smaller problems are eventually broken down in to the base case

## Recursion and the Stack of Function Calls
- Each recursive call to a function *normally* pushes another function call on the stack
    - We will see later a way to prevent this
- We can use this stack to trace through recursive calls

## Trace Example
```python 
def fact (n):
    return n * fact(n -1)
```

## Recursion
- Recursion is the preferred method in functional languages for processing a list element by element
- If possible, put the recursive call at the end to cause tail recursion
    - This allows a compiler to optimize a recursion function
 

In [ ]:
(define member
    (lambda (mem l)
       (cond
        ((null? l) #f)
        ((eq? mem (car l)) #t)
        (else (member mem (cdr l)))
       )
    )
)

In [ ]:
(member 3 `(1 2 3 4))

In [ ]:
(member 5 `(1 2 3 4 ))

In [ ]:
(define factorial 
      (lambda (n)
         (if
          (= 0 n)
          1
          (*
           (factorial (- n 1))
           n
          )
         )
      )
)

In [ ]:
(factorial 3000)

## Recursion Example
Write a recursive function the returns the length of a list
- Hint:
    - `null?` returns `#t` if a list is empty

In [ ]:
(len `(1 2 3 48 0 8))

## Recursion Practice
- Write a recursive function to return the maximum number found in a list
    - Should be called like `(max 1 2 3 4 5)`

## Tail Recursion
- In traditional recursion, each recursive call causes the currently executing code to be suspending, and placed on the call stack
    - This limits the recursion depth, and causes a stack overflow if too many recursive calls are made
- If we make the last call of the function the recursive call, we are effectively done with the calling function, and don't need to suspend it
    - Languages like Scheme take advantage of this, and keep a constant call stack of 1
    - Makes tail recursion equivalent to iteration in terms of speed and space

In [ ]:
(define tail_factorial
      (lambda (n result)
        (if (= n 0)
            result
            (tail_factorial (- n 1) (* n result))
         )
      )
)

In [ ]:
(define good_factorial
  (lambda (n)
    (tail_factorial n 1)
  )
)
(good_factorial 10)

## Tracing Tail Recursion
- Trace the `good_factorial` function with an input of 4

## Tail Recursion Example
- Write the following function using tail recursion
    - A function `range` that returns all the numbers between the two parameters, a and b

## Tail Recursion Example
- Write the following function using tail recursion
    - A function `rev` that returns a reversed version of the list passed to it

## Tail Recursion Practice
- Write the following function using tail recursion
    - A function `sum` that adds all the numbers in a list together

## Tail Recursion Practice
- Write the following function using tail recursion
    - A function `len` that finds the length of a list

## Letrec
- The scope of variables in a __let__ statement is the body of the statement
- What if we want to define a function in a __let__ statement recursively?
    - __letrec__ expands the scope slightly to include the value of the variable

In [ ]:
(letrec (
          (sum (lambda (ls)
                (if (null? ls)
                    0
                    (+ (car ls) (sum (cdr ls)))
                 )
               )
          )
         )
  (sum '(1 2 3 4 5))
)

## Let*
- Another variation of let is `let*`
    - This forces the evaluation of the variables in let in the order they are declared in
- `let*` allows variables declared earlier in the let block to be used when declared further variables
```scheme
(let*
      (
       (var1 value1)
       (var2 var1)
      )
      body
 )
```

In [ ]:
(let*
    ( 
      (n 10)
      (n_squared (* n n))
    )
   (display n_squared)
)

## Assignment
- Assignment is possible in Scheme, but there usually isn't a need for it
- __set!__ is used to update a variable

In [ ]:
(define old+ +)
(set! + -)
;(display (+ 2 4))
;(set! + old+)
;(display (old+ 2 4))

## Assignment Practice
Rewrite the function below to not use __set!__

In [ ]:
(define quadratic-formula
  (lambda (a b c)
    (let ((root1 0) (root2 0) (minusb 0) (radical 0) (divisor 0))
      (set! minusb (- b))
      (set! radical (sqrt (- (* b b) (* 4 (* a c)))))
      (set! divisor (* 2 a))
      (set! root1 (/ (+ minusb radical) divisor))
      (set! root2 (/ (- minusb radical) divisor))
      (cons root1 root2))))


(display (quadratic-formula 1 100 3 ))

## Closures
- Just like Lua, Scheme has closures
- I think they are actually clearer in Scheme
- General Layout:
```scheme
(define closureName (lambda ()
    (let ((varToClose val))
        (lambda ()
            update varToClose
        )
    )
)
```

## Closure Examples

In [ ]:
(define counter (lambda()
  (let ((count 0))
    (lambda ()
      (set! count (+ count 1))
      count
    )
    )
)
)

In [ ]:
(define c1 (counter))
(c1)

In [ ]:
(c1)

## Closure Practice
- Write a closure that takes in a variable, and returns a function that takes another variable 
    - This function will return greeting + name.
    - (define hi (closure "hello"))
    - (hi " world") results in "hello world"

## Objects
<small>From http://people.cs.aau.dk/~normark/prog3-03/html/notes/oop-scheme_themes-classes-objects-sec.html</small>

In [ ]:
(define (send message obj . par)
  (let ((method (obj message)))
    (apply method par)))

(define point (lambda(x y)
  (letrec ((getx    (lambda () x))
           (gety    (lambda () y))
           (add     (lambda (p) 
                      (point 
                       (+ x (send 'getx p))
                       (+ y (send 'gety p)))))
           (type-of (lambda () 'point))
          )
    (lambda (message)
      (cond ((eq? message 'getx) getx)
            ((eq? message 'gety) gety)
            ((eq? message 'add)  add)
            ((eq? message 'type-of) type-of)
            (else (error "Message not understood")))))))

In [ ]:
(define p1 (point 10 20))
(define p2 (point 1  2))
(display (send 'gety p1))
(newline)
(display (send 'getx p2))
(newline)

(define result (send 'add p1 p2))
(display (send 'gety result))
(newline)
(display (send 'getx result))